In [50]:
!pip install pdfplumber

  Using cached pillow-12.0.0-cp311-cp311-win_amd64.whl.metadata (9.0 kB)
  Using cached charset_normalizer-3.4.4-cp311-cp311-win_amd64.whl.metadata (38 kB)
  Using cached pycparser-2.23-py3-none-any.whl.metadata (993 bytes)
   ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
   ---------------------------------------- 5.6/5.6 MB 68.7 MB/s  0:00:00
Using cached charset_normalizer-3.4.4-cp311-cp311-win_amd64.whl (106 kB)
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ---------------------------------------- 3.5/3.5 MB 104.9 MB/s  0:00:00
Using cached pillow-12.0.0-cp311-cp311-win_amd64.whl (7.0 MB)
   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   ---------------------------------------- 3.1/3.1 MB 60.3 MB/s  0:00:00
Using cached pycparser-2.23-py3-none-any.whl (118 kB)

   ----- ---------------------------------- 1/8 [pycparser]
   ---------- ----------------------------- 2/8 [Pillow]
   ---------- -------------------------

In [ ]:
#creating the text corpus from the textbook ed3
from PyPDF2 import PdfReader
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path,'rb') as f:
        reader = PdfReader(f)
        for page_num, page in enumerate(reader.pages):
            text += page.extract_text() or ""
    return text 

pdf_path = "C:\Users\moham\Desktop\deeplearning\ed3book_aug25 no changes.pdf"
corpus_text = extract_text_from_pdf(pdf_path)

print("sample text: \n", corpus_text[:500])

Cannot set gray non-stroke color because /'P1' is an invalid float value


sample text: 
 Speech and Language Processing
An Introduction to Natural Language Processing,
Computational Linguistics, and Speech Recognition
with Language Models
Third Edition draft
Daniel Jurafsky
Stanford University
James H. Martin
University of Colorado at Boulder
Copyright ©2025. All rights reserved.
Draft of August 24, 2025. Comments and typos welcome!Summary of Contents
I LargeLanguageModels 1
1 Introduction................................................... 3
2 WordsandTokens.........................


In [24]:
import re 
import numpy as np
from collections import Counter, defaultdict 

def tokenize(text):
    tokens = re.findall(r"\b[a-zA-Z]+\b", text.lower())
    return tokens

tokenized_corpus = tokenize(corpus_text)
print(len(tokenized_corpus))

284753


In [25]:
#building vocab
vocab = sorted(set(tokenized_corpus))
word2idx = {w:i for i,w in enumerate(vocab)}
idx2word = {i:w for w,i in word2idx.items()}

print(len(vocab))

17830


In [26]:
#Unigram Probabilities , counting occurance of each word
counts = Counter(tokenized_corpus)
total = sum(counts.values())
unigram_probs = {w: counts[w]/total for w in vocab}

print("example probabilities")
for w,p in list(unigram_probs.items())[:10]:
    print(f"{w:>10} -> {p:.4f}")


example probabilities
         a -> 0.0285
        aa -> 0.0001
      aaaa -> 0.0000
      aaab -> 0.0000
aaacshicbzbls -> 0.0000
      aaai -> 0.0001
       aab -> 0.0000
       aac -> 0.0000
       aae -> 0.0000
  aaffects -> 0.0000


In [31]:
unigram_probs['hi']

3.160633952934649e-05

In [32]:
unigram_probs['language']

0.0039683515186846145

## N- Gram Probabilities

In [33]:
bigram_counts = defaultdict(lambda: defaultdict(int))
for i in range(len(tokenized_corpus)-1):
    w1, w2 = tokenized_corpus[i], tokenized_corpus[i+1]
    bigram_counts[w1][w2] += 1


bigram_probs = defaultdict(dict)
for w1 in bigram_counts:
    total = sum(bigram_counts[w1].values())
    for w2 in bigram_counts[w1]:
        bigram_probs[w1][w2] = bigram_counts[w1][w2] / total

In [36]:
bigram_probs['large']

{'language': 0.3758169934640523,
 'input': 0.0032679738562091504,
 'systems': 0.0032679738562091504,
 'number': 0.02287581699346405,
 'set': 0.029411764705882353,
 'corpus': 0.026143790849673203,
 'industrialized': 0.0032679738562091504,
 'problem': 0.006535947712418301,
 'lan': 0.00980392156862745,
 'enough': 0.0196078431372549,
 'objects': 0.0032679738562091504,
 'n': 0.00980392156862745,
 'leading': 0.0032679738562091504,
 'as': 0.00980392156862745,
 'neural': 0.0032679738562091504,
 'datasets': 0.0196078431372549,
 'overshooting': 0.0032679738562091504,
 'majority': 0.0032679738562091504,
 'difais': 0.0032679738562091504,
 'numbers': 0.0196078431372549,
 'is': 0.0032679738562091504,
 'weights': 0.0032679738562091504,
 'sets': 0.0032679738562091504,
 'because': 0.0032679738562091504,
 'values': 0.013071895424836602,
 'alan': 0.0032679738562091504,
 'network': 0.0032679738562091504,
 'perhaps': 0.0032679738562091504,
 'more': 0.0032679738562091504,
 'they': 0.0032679738562091504,
 'p

In [45]:
def build_ngram_counts(corpus, n=3):
    counts = defaultdict(lambda: defaultdict(int))
    for i in range(len(corpus)-n):
        ngram = tuple(corpus[i:i+n])
        w2 = corpus[i+n]
        counts[ngram][w2] += 1

    ngram_probs = defaultdict(dict)
    for ngram in counts:
        total = sum(counts[ngram].values())
        for w2 in counts[ngram]:
            ngram_probs[ngram][w2] = counts[ngram][w2] / total
    
    return ngram_probs 

ngram_proba = build_ngram_counts(tokenized_corpus,n=2)
    

In [ ]:
ngram_proba

{}

## Word2Vec
(context-target) pairs , with context being fixed and getting target windows from -n to +n window.

In [49]:
def gen_context_target_pairs(corpus, window_size=2):
    pairs=[]
    n = len(corpus)
    for i, target in enumerate(corpus):
        start = max(0,i-window_size)
        end = min(n, i+window_size+1)
        context_words = [corpus[j] for j in range(start,end) if j != i]
        for cw in context_words:
            pairs.append((cw, target))
    
    return pairs

cw_target_pairs = gen_context_target_pairs(tokenized_corpus)
print(len(cw_target_pairs))

#now when we want to do training for the context target pair, we will simply make it dictionary 
#with (cw,target) as the key and their value is [1xD] vector or any vector which is initialiased
#randomly then , we will keep updating the value as 
#but for this approach at end we will get a pairs set of size vocab_size*vocab_size for only 2 pair context
#and this value will keep increasing in permutation order.

1139006
